# Memory (대화 내용 기억)

- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다.
- 이전 대화내용을 계속 프롬프트에 주입해야 함

1. Short-Term Memory
단기기억: 한 대화 세션에 대한 기억
2. Long-Term Memory
장기기억: 전체 세션에서 추출한 중요한 정보

## Memory 구동방식
1. 메모리는 기본적으로 몯느 ㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏㅏ\

In [3]:
# 실제 gpt와 대화하는 모습으로 만들기

input_msg = input()
while input_msg in ('quit','정지','그만'):
    pass

## `ConversationBufferMemory`
- 메시지 저장 -> 변수에서 추출 가능

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context(
    inputs = {
        'human':'안녕 나는 비대면 은행 계좌를 개설하고 싶어. 어찌해야함?'
    },
    outputs = {
        'ai':'좋아. 우선 신분증을 준비해 줘'
    }
)

print(memory.load_memory_variables({}))

memory.save_context(
    inputs = {'human': '좋아. 준비했어'},
    outputs={'ai':'앞면을 잘 촬용해서 업로드 해줘.'}
)
print(memory.load_memory_variables({}))

C:\Users\mingyu\AppData\Local\Temp\ipykernel_15220\61187545.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()


{'history': 'Human: 안녕 나는 비대면 은행 계좌를 개설하고 싶어. 어찌해야함?\nAI: 좋아. 우선 신분증을 준비해 줘'}

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationChain

# temperature => 랜덤성 (0 ~ 1)
llm = ChatOpenAI(model = 'gpt-4.1-nano',temperature = 0)

prompt = ChatPromptTemplate.from_messages(
    [
        ('system','너는 친절한 어시스턴트야'),
        ('human','{query}'),

    ]
)
conversation = ConversationChain(
    llm = llm,
    memory = ConversationBufferMemory(),
    prompt = prompt
)

response = conversation.invoke({'query':'안녕 계좌를 만들고 싶어'})

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = 'gpt-4.1-nano',temperature=0)
prompt = ChatPromptTemplate(
    [
        ("system",'넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name= 'chat_history'),
        ('human','{input}'),
    ]
)
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 벼ㄴ수는 {}디. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})


In [ ]:
chain = runnable | prompt | llm
res = chain.invoke({'input':'만나서 반가워'})

In [ ]:
memory.save_context({
    'human':'만나서 반가워',
    
})

In [ ]:
my_message = '내 이름은 유태영이야'
res = chain.invoke({'input':my_message})
memory.save_context(
    {'human'}
)